In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from scipy.signal import hilbert
import numpy as np
from matplotlib.colors import Normalize
from matplotlib.collections import LineCollection

In [2]:
def get_stft(data_flat, fs=1000, nperseg=64):
    # Calculate the instantaneous frequency using the Hilbert transform
    analytic_signal = hilbert(data_flat, N = nperseg)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    # Fix the instantaneous frequency calculation
    dt = 1/fs  # Time step
    instantaneous_frequency = np.abs(np.diff(instantaneous_phase) / (2.0 * np.pi * dt))
    # Pad the instantaneous_frequency to match the length of data_flat
    instantaneous_frequency = np.append(instantaneous_frequency, instantaneous_frequency[-1])
    return instantaneous_frequency

In [3]:
data_path = Path('./data/Data6Sim/Incipient Fault')

In [4]:
os.listdir(data_path)

['Disturbance', 'cable', 'Normal']

In [5]:
# Get all .xlsx files from the cable/current directory
cable_current_path = data_path / 'cable' / 'current'
cable_voltage_path = data_path / 'cable' / 'voltage'
disturbance_current_path = data_path / 'disturbance' / 'current'
disturbance_voltage_path = data_path / 'disturbance' / 'voltage'
normal_current_path = data_path / 'normal' / 'current'
normal_voltage_path = data_path / 'normal' / 'voltage'

cable_current_xlsx_files = list(cable_current_path.glob('*.xlsx'))
cable_voltage_xlsx_files = list(cable_voltage_path.glob('*.xlsx'))
disturbance_current_xlsx_files = list(disturbance_current_path.glob('*.xlsx'))
disturbance_voltage_xlsx_files = list(disturbance_voltage_path.glob('*.xlsx'))
normal_current_xlsx_files = list(normal_current_path.glob('*.xlsx'))
normal_voltage_xlsx_files = list(normal_voltage_path.glob('*.xlsx'))
# Sort the Excel files alphabetically for better organization
cable_current_xlsx_files.sort(key=lambda x: x.name)
cable_voltage_xlsx_files.sort(key=lambda x: x.name)
disturbance_current_xlsx_files.sort(key=lambda x: x.name)
disturbance_voltage_xlsx_files.sort(key=lambda x: x.name)
normal_current_xlsx_files.sort(key=lambda x: x.name)
normal_voltage_xlsx_files.sort(key=lambda x: x.name)

In [6]:
def add_to_plot(data_flat, stft_data, cmap, norm, ax):
    time_index = np.arange(data_flat.shape[0])
    
    # Create points for the line segments
    points = np.array([time_index, data_flat]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    # Create a colored line collection
    lc = LineCollection(segments, cmap=cmap, norm=norm)
    
    lc.set_array(stft_data)
    lc.set_linewidth(6)

    # Add the collection to the plot
    line = ax.add_collection(lc)


In [7]:
cmap = plt.cm.hot  # viridis provides good contrast and is perceptually uniform

In [8]:
for i in range(0, len(cable_current_xlsx_files), 3):
    df_A_cable_current = pd.read_excel(cable_current_xlsx_files[i])
    df_B_cable_current = pd.read_excel(cable_current_xlsx_files[i+1])
    df_C_cable_current = pd.read_excel(cable_current_xlsx_files[i+2])

    data_A_cable_current = df_A_cable_current.to_numpy()
    data_B_cable_current = df_B_cable_current.to_numpy()
    data_C_cable_current = df_C_cable_current.to_numpy()

    A_cable_current_flat = data_A_cable_current[:, 0].flatten()
    B_cable_current_flat = data_B_cable_current[:, 0].flatten()
    C_cable_current_flat = data_C_cable_current[:, 0].flatten()

    current_flatten_min = min(A_cable_current_flat.min(), B_cable_current_flat.min(), C_cable_current_flat.min())
    current_flatten_max = max(A_cable_current_flat.max(), B_cable_current_flat.max(), C_cable_current_flat.max())

    stft_A_cable_current = get_stft(A_cable_current_flat)
    stft_B_cable_current = get_stft(B_cable_current_flat)
    stft_C_cable_current = get_stft(C_cable_current_flat)

    current_stft_min = min(stft_A_cable_current.min(), stft_B_cable_current.min(), stft_C_cable_current.min())
    current_stft_max = max(stft_A_cable_current.max(), stft_B_cable_current.max(), stft_C_cable_current.max())

    # Plot the STFT
    fig, ax = plt.subplots(figsize=(6, 6))
    fig.patch.set_facecolor('grey')  # 设置整个图像的背景色
    ax.set_facecolor('white')  
    # Turn off the axes frame
    ax.set_frame_on(False)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)


    # Plot 1: Signal with colored frequency
    # Create a normalized color map
    norm = Normalize(vmin=current_stft_min, 
                    vmax=current_stft_max)
    add_to_plot(A_cable_current_flat, stft_A_cable_current, cmap, norm, ax)
    add_to_plot(B_cable_current_flat, stft_B_cable_current, cmap, norm, ax)
    add_to_plot(C_cable_current_flat, stft_C_cable_current, cmap, norm, ax)

    time_index = np.arange(data_A_cable_current.shape[0])
    # Set plot limits
    ax.set_xlim(time_index.min(), time_index.max())
    ax.set_ylim(current_flatten_min - 0.1, current_flatten_max + 0.1)

    plt.savefig(f'./figures/cable_current_{i//3}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)
    # plt.show()

In [9]:
for i in range(0, len(disturbance_current_xlsx_files), 3):
    df_A_disturbance_current = pd.read_excel(disturbance_current_xlsx_files[i])
    df_B_disturbance_current = pd.read_excel(disturbance_current_xlsx_files[i+1])
    df_C_disturbance_current = pd.read_excel(disturbance_current_xlsx_files[i+2])

    data_A_disturbance_current = df_A_disturbance_current.to_numpy()
    data_B_disturbance_current = df_B_disturbance_current.to_numpy()
    data_C_disturbance_current = df_C_disturbance_current.to_numpy()

    A_disturbance_current_flat = data_A_disturbance_current[:, 0].flatten()
    B_disturbance_current_flat = data_B_disturbance_current[:, 0].flatten()
    C_disturbance_current_flat = data_C_disturbance_current[:, 0].flatten()

    current_flatten_min = min(A_disturbance_current_flat.min(), B_disturbance_current_flat.min(), C_disturbance_current_flat.min())
    current_flatten_max = max(A_disturbance_current_flat.max(), B_disturbance_current_flat.max(), C_disturbance_current_flat.max())

    stft_A_disturbance_current = get_stft(A_disturbance_current_flat)
    stft_B_disturbance_current = get_stft(B_disturbance_current_flat)
    stft_C_disturbance_current = get_stft(C_disturbance_current_flat)

    current_stft_min = min(stft_A_disturbance_current.min(), stft_B_disturbance_current.min(), stft_C_disturbance_current.min())
    current_stft_max = max(stft_A_disturbance_current.max(), stft_B_disturbance_current.max(), stft_C_disturbance_current.max())

    # Plot the STFT
    fig, ax = plt.subplots(figsize=(6, 6))
    fig.patch.set_facecolor('grey')  # 设置整个图像的背景色
    ax.set_facecolor('white')  
    # Turn off the axes frame
    ax.set_frame_on(False)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    # Plot 1: Signal with colored frequency
    # Create a normalized color map
    norm = Normalize(vmin=current_stft_min, 
                    vmax=current_stft_max)
    add_to_plot(A_disturbance_current_flat, stft_A_disturbance_current, cmap, norm, ax)
    add_to_plot(B_disturbance_current_flat, stft_B_disturbance_current, cmap, norm, ax)
    add_to_plot(C_disturbance_current_flat, stft_C_disturbance_current, cmap, norm, ax)

    time_index = np.arange(data_A_disturbance_current.shape[0])
    # Set plot limits
    ax.set_xlim(time_index.min(), time_index.max())
    ax.set_ylim(current_flatten_min - 0.1, current_flatten_max + 0.1)

    plt.savefig(f'./figures/disturbance_current_{i//3}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)
    # plt.show()

In [10]:
for i in range(0, len(normal_current_xlsx_files), 3):
    df_A_normal_current = pd.read_excel(normal_current_xlsx_files[i])
    df_B_normal_current = pd.read_excel(normal_current_xlsx_files[i+1])
    df_C_normal_current = pd.read_excel(normal_current_xlsx_files[i+2])

    data_A_normal_current = df_A_normal_current.to_numpy()
    data_B_normal_current = df_B_normal_current.to_numpy()
    data_C_normal_current = df_C_normal_current.to_numpy()

    A_normal_current_flat = data_A_normal_current[:, 0].flatten()
    B_normal_current_flat = data_B_normal_current[:, 0].flatten()
    C_normal_current_flat = data_C_normal_current[:, 0].flatten()

    current_flatten_min = min(A_normal_current_flat.min(), B_normal_current_flat.min(), C_normal_current_flat.min())
    current_flatten_max = max(A_normal_current_flat.max(), B_normal_current_flat.max(), C_normal_current_flat.max())

    stft_A_normal_current = get_stft(A_normal_current_flat)
    stft_B_normal_current = get_stft(B_normal_current_flat)
    stft_C_normal_current = get_stft(C_normal_current_flat)

    current_stft_min = min(stft_A_normal_current.min(), stft_B_normal_current.min(), stft_C_normal_current.min())
    current_stft_max = max(stft_A_normal_current.max(), stft_B_normal_current.max(), stft_C_normal_current.max())

    # Plot the STFT
    fig, ax = plt.subplots(figsize=(6, 6))
    
    fig.patch.set_facecolor('grey')  # 设置整个图像的背景色
    ax.set_facecolor('white')   

    # Turn off the axes frame
    ax.set_frame_on(False)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)  

    # Plot 1: Signal with colored frequency
    # Create a normalized color map
    norm = Normalize(vmin=current_stft_min, 
                    vmax=current_stft_max)
    add_to_plot(A_normal_current_flat, stft_A_normal_current, cmap, norm, ax)
    add_to_plot(B_normal_current_flat, stft_B_normal_current, cmap, norm, ax)
    add_to_plot(C_normal_current_flat, stft_C_normal_current, cmap, norm, ax)

    time_index = np.arange(data_A_normal_current.shape[0])
    # Set plot limits
    ax.set_xlim(time_index.min(), time_index.max())
    ax.set_ylim(current_flatten_min - 0.1, current_flatten_max + 0.1)

    plt.savefig(f'./figures/normal_current_{i//3}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)
    # plt.show()